In [1]:
import glob
import os
import numpy as np
from skimage import io, transform, exposure, restoration, filters
from skimage.measure import regionprops
from scipy import ndimage
import matplotlib.pyplot as plt
import tqdm
import pims
import panel as pn

from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Spectral11
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.models import Range1d

import pandas as pd


import bokeh.io
from bokeh.plotting import figure, show


from bokeh.io import output_notebook
output_notebook()

pn.extension()

Loading BokehJS ...

In [24]:
# I saved the four test videos in data_dir 
#home_dir = os.path.expanduser('~')
data_dir = os.path.join('/Users/scliu/Documents/AM_imaging/data/pre_print_data/1228_phase/')
data_list = sorted(glob.glob(data_dir + '/*.tif'))
file_list = sorted(glob.glob('/Users/scliu/Documents/AM_imaging/data/pre_print_data/1228_phase/155min_MT_circle_1.tif'))

In [25]:
# idx indexes the list of videos
ims = io.imread('/Users/scliu/Documents/AM_imaging/data/pre_print_data/1228_phase/155min_1ul_6-1.tif')
output_shape = (512,512)
output_len = 200

In [4]:
# Most skimage functions can only take one image at a time
# We need to implement loops several times because we have videos
def apply_function_to_ims(ims, function, new_shape = None, **kwargs):
    if not new_shape:
        new_shape = ims.shape
    output_ims = np.zeros(new_shape).astype('float32')
    for i, _ in enumerate(tqdm.tqdm(ims)): # loop over images
        output_ims[i,:,:]  = function(ims[i,:,:], **kwargs)
    return output_ims

# skimage thresholding functions only give you the scalar value of the threshold
# we need to create the binary masks using the threshold
def thresholding(im, thresholding_function = None):
    return (im > thresholding_function(im)).astype('float32')

In [5]:
ims[:200, 0, :,:].shape

(200, 2048, 2048)

In [9]:
# Scale the video so it is easier to work with
X_scaled = apply_function_to_ims(
    ims[:output_len, 0, :,:], 
    transform.resize, 
    new_shape = (ims.shape[0], *output_shape), 
    output_shape = output_shape,
)


100%|█████████████████████████████████████████| 200/200 [00:18<00:00, 10.72it/s]


In [10]:
# We will use the binarized illumination region image to set to 0 all the pixels in our
# aster video that are outside of illumination region.
illumination_mask = transform.resize(ims[0,1,:,:], output_shape)
# I found this threshold (.00175) by looking at histogram of the illumination region image.
illumination_mask = (illumination_mask > 0.0016).astype('float32')
unique, counts = np.unique(illumination_mask, return_counts=True)
dict(zip(unique, counts))
X_scaled = X_scaled * illumination_mask
del ims

In [11]:
# Normalize the intensity
X_scaled_normalized = X_scaled/np.sum(X_scaled)

In [12]:
# find the image moments of each frame
centers = [ndimage.measurements.center_of_mass(frame) for frame in X_scaled_normalized]
centers = np.array(centers)

/opt/anaconda3/envs/active_matters/lib/python3.9/site-packages/scipy/ndimage/measurements.py:1390: RuntimeWarning: invalid value encountered in double_scalars
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


In [13]:
# calculate variances and standard deviation
x = np.arange(X_scaled_normalized.shape[2])
y = np.arange(X_scaled_normalized.shape[1])
variances = np.sum(((x[None, None, :] - centers[:, 1, None, None]) ** 2 + (y[None, :, None] - centers[:, 0, None, None]) ** 2) * X_scaled_normalized, axis=(1,2))
SD = np.sqrt(variances)

In [14]:
# define x and y
x = (np.arange(len(X_scaled_normalized)) + 1) * 10 / 60
y = SD

In [15]:
#plot the data
p = figure(plot_width=800, plot_height=300, 
            x_axis_label='time(min)', 
            y_axis_label='image standard deviation (μm)')

p.x_range = Range1d(0, 20)

p.line(x= x ,
        y = y)

show(p)


In [26]:
data_list

['/Users/scliu/Documents/AM_imaging/data/pre_print_data/1228_phase/155min_1ul_6-1.tif',
 '/Users/scliu/Documents/AM_imaging/data/pre_print_data/1228_phase/6_min_1ul_6-1.tif']

In [27]:

# Create an empty dataframe to store the values
df = pd.DataFrame(columns=['video_name', 'variances', 'SD'])

# Loop through the list of videos
for data in data_list:
    # Get the video name
    video_name = os.path.basename(data)
    
    # Load the video
    ims = io.imread(data)
    
    # Process the video
    X_scaled = apply_function_to_ims(
        ims[:output_len, 0, :,:], 
        transform.resize, 
        new_shape = (ims.shape[0], *output_shape), 
        output_shape = output_shape,
    )
    X_scaled = X_scaled * illumination_mask
    X_scaled_normalized = X_scaled/np.sum(X_scaled)
    x = np.arange(X_scaled_normalized.shape[2])
    y = np.arange(X_scaled_normalized.shape[1])
    centers = [ndimage.measurements.center_of_mass(frame) for frame in X_scaled_normalized]
    centers = np.array(centers)
    variances = np.sum(((x[None, None, :] - centers[:, 1, None, None]) ** 2 + (y[None, :, None] - centers[:, 0, None, None]) ** 2) * X_scaled_normalized, axis=(1,2))
    SD = np.sqrt(variances)
    
    # Store the values in the dataframe
    df = df.append({'video_name': video_name, 'variances': variances, 'SD': SD}, ignore_index=True)
    
    del ims
    del X_scaled
    del X_scaled_normalized
# Print the dataframe
print(df)


100%|█████████████████████████████████████████| 200/200 [00:19<00:00, 10.36it/s]
/opt/anaconda3/envs/active_matters/lib/python3.9/site-packages/scipy/ndimage/measurements.py:1390: RuntimeWarning: invalid value encountered in double_scalars
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer
/var/folders/p3/mdh493ks43l04slykq6gwbx80000gn/T/ipykernel_17605/876385595.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_name': video_name, 'variances': variances, 'SD': SD}, ignore_index=True)
100%|█████████████████████████████████████████| 200/200 [00:19<00:00, 10.52it/s]
/opt/anaconda3/envs/active_matters/lib/python3.9/site-packages/scipy/ndimage/measurements.py:1390: RuntimeWarning: invalid value encountered in double_scalars
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


           video_name                                          variances  \
0  155min_1ul_6-1.tif  [53.48737552632993, 79.43056143694817, 79.3138...   
1   6_min_1ul_6-1.tif  [55.53364768805323, 82.49544840780376, 82.5182...   

                                                  SD  
0  [7.313506376993865, 8.912382478156342, 8.90583...  
1  [7.452090155657889, 9.082700501932438, 9.08395...  


/var/folders/p3/mdh493ks43l04slykq6gwbx80000gn/T/ipykernel_17605/876385595.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_name': video_name, 'variances': variances, 'SD': SD}, ignore_index=True)


In [28]:
df

,video_name,variances,SD
0,155min_1ul_6-1.tif,"[53.48737552632993, 79.43056143694817, 79.3138...","[7.313506376993865, 8.912382478156342, 8.90583..."
1,6_min_1ul_6-1.tif,"[55.53364768805323, 82.49544840780376, 82.5182...","[7.452090155657889, 9.082700501932438, 9.08395..."
